In [1]:
import pandas as pd

df = pd.read_excel('synthetic_products 2.xlsx')
df.head(3).style.format({"YTD vs last year": lambda x: f"{x:.2%}" if pd.notna(x) else "nan%"})

,Product,2024 YTB,2025 YTB,2026,YTD vs last year
0,Alpha Fund Europe,112.806000,125.340000,132.780000,0.06%
1,Alpha Fund US,88.308000,98.120000,101.450000,0.03%
2,Alpha Fund Asia,78.750000,87.500000,83.270000,-0.45%


In [2]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill, Font
from openpyxl.utils import get_column_letter


def export_df_to_colored_excel(
    df: pd.DataFrame,
    output_path: str,
    sheet_name: str = "Sheet1",
    wb: openpyxl.Workbook = None,
) -> openpyxl.Workbook:
    """Exporte le DataFrame dans un onglet Excel coloré selon 'YTD vs last year' (float).

    Paramètres
    ----------
    df          : DataFrame à exporter
    output_path : chemin du fichier .xlsx de sortie
    sheet_name  : nom de l'onglet (défaut : 'Sheet1')
    wb          : workbook existant à compléter ; si None, un nouveau est créé

    Retourne le workbook (pour enchaîner plusieurs onglets).

    Normalisation symétrique sur [-1, 1] :
        -1  → rouge vif (FF0000) | 0 → blanc (FFFFFF) | +1 → vert vif (00C800)
    Intensité minimale de 30% pour toute valeur non nulle.
    """

    col = "YTD vs last year"
    if col not in df.columns:
        raise ValueError(f"La colonne '{col}' est absente du DataFrame.")

    # 1. Colonne déjà en float (0.0594, -0.0484, NaN…)
    perc = pd.to_numeric(df[col], errors="coerce")

    # 2. Normalisation symétrique
    abs_max = perc.abs().max()
    if pd.isna(abs_max) or abs_max == 0:
        normalized = perc.fillna(0) * 0
    else:
        normalized = (perc / abs_max).clip(-1, 1)

    # 3. Score t ∈ [-1, 1] → couleur hex
    MIN_INTENSITY = 0.30

    def t_to_hex(t: float) -> str:
        if pd.isna(t):
            return "FFFFFF"
        t = float(t)
        if t == 0:
            return "FFFFFF"
        intensity = MIN_INTENSITY + (1 - MIN_INTENSITY) * abs(t)
        if t < 0:
            r, g, b = 255, int(255 * (1 - intensity)), int(255 * (1 - intensity))
        else:
            r = int(255 * (1 - intensity))
            g = int(255 * (1 - intensity) + 200 * intensity)
            b = int(255 * (1 - intensity))
        return f"{r:02X}{g:02X}{b:02X}"

    # 4. Créer ou réutiliser le workbook
    if wb is None:
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = sheet_name
    else:
        ws = wb.create_sheet(title=sheet_name)

    ytd_col_idx = list(df.columns).index(col) + 1

    for c_idx, col_name in enumerate(df.columns, start=1):
        cell = ws.cell(row=1, column=c_idx, value=col_name)
        cell.font = Font(bold=True)
        ws.column_dimensions[get_column_letter(c_idx)].width = 22

    for r_idx, (_, row) in enumerate(df.iterrows(), start=2):
        hex_color = t_to_hex(normalized.iloc[r_idx - 2])
        fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")

        for c_idx, val in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx, value=val)
            cell.fill = fill
            if c_idx == ytd_col_idx:
                cell.number_format = "0.00%"

    wb.save(output_path)
    print(f"✅ Onglet '{sheet_name}' sauvegardé dans {output_path}")
    return wb


# ── Exemple multi-onglets ──────────────────────────────────────────────────
# df1 et df2 sont deux DataFrames distincts
# wb = export_df_to_colored_excel(df1, "multi.xlsx", sheet_name="Produits A")
# export_df_to_colored_excel(df2, "multi.xlsx", sheet_name="Produits B", wb=wb)

# ── Exemple onglet unique (comportement inchangé) ──────────────────────────
wb = export_df_to_colored_excel(df, "df_colored.xlsx", sheet_name="Produits 4")
export_df_to_colored_excel(df, "df_colored.xlsx", sheet_name="Produits 3", wb=wb)

✅ Onglet 'Produits 4' sauvegardé dans df_colored.xlsx
✅ Onglet 'Produits 3' sauvegardé dans df_colored.xlsx


In [3]:
import plotly.graph_objects as go


def plot_bar_comparison(
    df: pd.DataFrame,
    product_col: str = "Product",
    ytd_col: str = "YTD vs last year",
    sort_by: str = None,
    title: str = "📊 Comparaison annuelle par produit (annotation = YTD)",
    height: int = 580,
    width: int = 1400,
) -> go.Figure:
    """Affiche un bar chart groupé (palette verte BNP) pour toutes les colonnes
    numériques d'années détectées automatiquement dans le DataFrame.

    Paramètres
    ----------
    df          : DataFrame source
    product_col : colonne contenant le nom des produits  (défaut : 'Product')
    ytd_col     : colonne YTD en décimal                 (défaut : 'YTD vs last year')
    sort_by     : colonne de tri                         (défaut : dernière année détectée)
    title       : titre du graphique
    height      : hauteur en pixels                      (défaut : 580)
    width       : largeur en pixels                      (défaut : 1400)

    Retourne
    --------
    go.Figure
    """
    year_cols = [
        c for c in df.columns
        if c not in [product_col, ytd_col]
        and pd.api.types.is_numeric_dtype(df[c])
    ]
    if not year_cols:
        raise ValueError("Aucune colonne numérique d'année détectée dans le DataFrame.")

    sort_col = sort_by or year_cols[-1]
    df_bar = df.sort_values(sort_col, ascending=False).copy()

    if ytd_col in df_bar.columns:
        df_bar["_ytd_pct"]   = df_bar[ytd_col] * 100
        df_bar["_ytd_label"] = df_bar["_ytd_pct"].apply(
            lambda x: f"{x:+.2f}%" if pd.notna(x) else ""
        )
        df_bar["_ytd_color"] = df_bar["_ytd_pct"].apply(
            lambda x: "#16a34a" if (pd.notna(x) and x >= 0) else "#dc2626"
        )
    else:
        df_bar["_ytd_label"] = ""
        df_bar["_ytd_color"] = "#94a3b8"

    # Palette verte BNP dégradée selon le nombre d'années
    bnp_greens = ["#a8d5b5", "#3d9e5f", "#00573f", "#003d2b"]
    palette = (bnp_greens * ((len(year_cols) // len(bnp_greens)) + 1))[:len(year_cols)]

    fig = go.Figure()
    for col, color in zip(year_cols, palette):
        fig.add_trace(go.Bar(
            x=df_bar[product_col],
            y=df_bar[col],
            name=col,
            marker_color=color,
            marker_line_color=color,
            marker_line_width=0.5,
            hovertemplate=f"<b>%{{x}}</b><br>{col} : %{{y:.2f}}<extra></extra>",
        ))

    for _, row in df_bar.iterrows():
        if row["_ytd_label"]:
            y_max = max(row[c] for c in year_cols if pd.notna(row[c]))
            fig.add_annotation(
                x=row[product_col],
                y=y_max + 3,
                text=row["_ytd_label"],
                showarrow=False,
                font=dict(size=8.5, color=row["_ytd_color"], family="Arial"),
            )

    fig.update_layout(
        title=dict(text=title, font=dict(size=17, color="#1e293b"), x=0.02),
        barmode="group",
        height=height,
        width=width,
        margin=dict(l=60, r=40, t=80, b=160),
        plot_bgcolor="white",
        paper_bgcolor="white",
        xaxis=dict(tickangle=-45, tickfont=dict(size=10), gridcolor="#e2e8f0"),
        yaxis=dict(title="Valeur (base 100)", gridcolor="#e2e8f0", zeroline=False),
        legend=dict(orientation="h", yanchor="bottom", y=1.01,
                    xanchor="right", x=1, font=dict(size=12)),
        font=dict(family="Arial, sans-serif", color="#334155"),
        bargap=0.25,
        bargroupgap=0.05,
    )
    return fig


# ── Appel ────────────────────────────────────────────────────────────────────
plot_bar_comparison(df)

In [ ]:
import io
from openpyxl.drawing.image import Image as XLImage

OUTPUT_PATH = "df_colored.xlsx"
SHEET_NAME  = "Graphique"

# 1. Générer le graphique
fig = plot_bar_comparison(df)

# 2. Exporter en PNG haute résolution dans un buffer mémoire
img_bytes = fig.to_image(format="png", scale=2)
image_stream = io.BytesIO(img_bytes)

# 3. Ouvrir le workbook existant et ajouter l'onglet graphique
wb = openpyxl.load_workbook(OUTPUT_PATH)

if SHEET_NAME in wb.sheetnames:
    del wb[SHEET_NAME]
ws = wb.create_sheet(title=SHEET_NAME)

xl_img = XLImage(image_stream)
xl_img.anchor = "B2"
ws.add_image(xl_img)

wb.save(OUTPUT_PATH)
print(f"✅ Graphique inséré dans l'onglet '{SHEET_NAME}' de {OUTPUT_PATH}")

✅ Graphique inséré dans l'onglet 'Graphique' de df_colored.xlsx
